In [2]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

# Pitching Airfoil 

In [3]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from time import time
from functools import partial
from jax import jit

# Dynamical Systems

In [4]:
from ICARUS.Systems.first_order_system import NonLinearSystem
from ICARUS.Systems.second_order_system import SecondOrderSystem
from ICARUS.Systems.integrate import BackwardEulerIntegrator, ForwardEulerIntegrator, RK4Integrator, RK45Integrator, CrankNicolsonIntegrator, GaussLegendreIntegrator, NewmarkIntegrator

In [25]:
def plot_results(x_data, t_data):
    # Plot the results
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    for key in list(x_data.keys()):
        # Compute cap so that we plot 1000 points at most
        cap = max(1, int(len(t_data[key]) / 1000))
        ax.plot(t_data[key][::cap], x_data[key][:,0][::cap], label=key)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Displacement (m)")
    ax.legend()


def test_all_integrators(system,x0, t0, t_end, dt0, compare_with_scipy = False):
    if isinstance(system, SecondOrderSystem):
        system2 = system.convert_to_first_order()
        newmark = NewmarkIntegrator(dt0, system, gamma=0.5, beta=0.25)
    else:
        system2 = system
    
    integrator_rk4 = RK4Integrator(dt0, system2)
    integrator_feuler = ForwardEulerIntegrator(dt0, system2)
    integrator_beuler = BackwardEulerIntegrator(dt0, system2)
    integrator_rk45 = RK45Integrator(dt0, system2)
    integrator_gauss_legendre = GaussLegendreIntegrator(dt0, system2, n = 4)
    integrator_crank_nicolson = CrankNicolsonIntegrator(dt0, system2)

    integrators = [
        integrator_feuler,
        integrator_rk4,
        # integrator_rk45,
        integrator_beuler,
        # integrator_gauss_legendre,
        # integrator_crank_nicolson
     ]
    
    if isinstance(system, SecondOrderSystem):
        integrators.append(newmark)

    # Simulate the system using all the integrators
    x_data = {}
    t_data = {}


    for integrator in integrators:
        print(f"Simulating using {integrator.name} integrator")
        time_s = time()
        t_data[integrator.name], x_data[integrator.name] = integrator.simulate(x0, t0, t_end)
        time_e = time()
        print(f"\tSimulated using {integrator.name} integrator")
        print(f"\tTime taken: {time_e - time_s} seconds")

    if compare_with_scipy:
        print("Simulating using scipy RK45")
        time_s = time()
        sol = solve_ivp(system, [t0, t_end], x0, method='RK45', t_eval=np.linspace(0, t_end, 1000),  rtol=1e-6, atol=1e-6)
        t_data["scipy_rk45"] = sol.t
        x_data["scipy_rk45"] = sol.y.T
        time_e = time()
        print(f"\tSimulated using scipy RK45")
        print(f"\tTime taken: {time_e - time_s} seconds")

    plot_results(x_data, t_data)


# Simple Mass-Damper System

In [6]:
# Define a simple m-c-k system
m = 1.0
c = 0.1
k = 1.0


def f(t:float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array([
        x[1],                            # x' = v
        -c /m * x[1] - k/m * x[0]        # v' = a = -c/m * v - k/m * x
    ])

# Create the system
system = NonLinearSystem(f)

# Test the integrators
test_all_integrators(system, jnp.array([1.0, 0.0]), 0.0, 100.0, 0.0001, compare_with_scipy = True)

Simulating using Forward Euler integrator
	Simulated using Forward Euler integrator
	Time taken: 0.3398106098175049 seconds
Simulating using RK4 integrator
	Simulated using RK4 integrator
	Time taken: 0.2039628028869629 seconds
Simulating using Backward Euler integrator
	Simulated using Backward Euler integrator
	Time taken: 0.7166538238525391 seconds
Simulating using scipy RK45
	Simulated using scipy RK45
	Time taken: 0.18001079559326172 seconds


# Second Order Systems

In [33]:
# Define a 2nd order system 
m1 = 1.0
c1 = 0.1
k1 = 1.0

m2 = 1.0
c2 = 0.1
k2 = 1.0

def M(t,x):
    return jnp.array(
    [
        [m1, 0],
        [0, m2]
    ])
# M = jnp.array([m])
def C(t,x):
    return jnp.array(
    [
        [0.023, 1.024], #[c1, 0],
        [-0.364, 3.31]  #[0, c2]
    ])
# C = jnp.array([c])

def f_int(t,x):
    return jnp.array(
    [
        [1.97, 0.034],  #[k1, -k1],
        [0.034, 3.95]   #[-k1, k1 + k2]
    ])
# f_int = jnp.array([k])

def f_ext(t: float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array([
        0.078,# 0.0,
        10*0.466*jnp.sin(t)
    ])
# f_ext = lambda t, x: jnp.array([0.0])

system = SecondOrderSystem(M, C, f_int, f_ext)

In [35]:
# Test the integrators
test_all_integrators(system, jnp.array([0.0, 0.0, 0.0, 0.0]), 0.0, 100.0, 1e-4, compare_with_scipy = True)

Simulating using Forward Euler integrator
	Simulated using Forward Euler integrator
	Time taken: 0.6067991256713867 seconds
Simulating using RK4 integrator
	Simulated using RK4 integrator
	Time taken: 1.5589611530303955 seconds
Simulating using Backward Euler integrator
	Simulated using Backward Euler integrator
	Time taken: 1.4609050750732422 seconds
Simulating using Newmark integrator
	Simulated using Newmark integrator
	Time taken: 0.4593162536621094 seconds
Simulating using scipy RK45
	Simulated using scipy RK45
	Time taken: 0.23194241523742676 seconds


# Node

In [36]:
class Node:
    idx = 0
    var_global_idx = {}
    """
    A node represents a point that has some degree of freedom (DOF) associated with it.
    Additionally it can have boundary conditions applied to it.
    """
    def __init__(self, degree_of_freedom: dict[str, float], boundary_conditions = None):
        self.dof = len(degree_of_freedom.keys())
        self.variables = degree_of_freedom
        self.boundary_conditions = boundary_conditions

        # Node index
        self.index = Node.idx
        Node.idx += 1

        # For each degree of freedom, assign a global index if it is not already assigned
        for key in degree_of_freedom.keys():
            if key not in Node.var_global_idx:
                Node.var_global_idx[key] = len(Node.var_global_idx)
        

    def __str__(self):
        return f"Node {self.index} with {self.dof} degrees of freedom"
    
    @staticmethod
    def get_global_idx(key)-> int:
        return Node.var_global_idx[key]
    
    @staticmethod
    def get_global_dof_name(idx):
        for key, value in Node.var_global_idx.items():
            if value == idx:
                return key
        return None
    
# Create a simple 3 node system
node1 = Node({"x1": 0.0, "y1": 0.0, "theta1": 0.0})
node2 = Node({"x2": 1.0, "y2": 0.0, "theta2": 0.0})
node3 = Node({"x3": 20.0, "y3": 0.0, "theta3": 0.0})


In [37]:
Node.var_global_idx

{'x1': 0,
 'y1': 1,
 'theta1': 2,
 'x2': 3,
 'y2': 4,
 'theta2': 5,
 'x3': 6,
 'y3': 7,
 'theta3': 8}

In [38]:
class NodeConnection:
    """
    A node connection represents how two nodes are connected to each other 
    and how they are connected to the global coordinate system.    
    """
    def __init__(self, node_1: Node, node_2: Node, phi: float):
        self.node_1 = node_1
        self.node_2 = node_2
        self.nodes = [node_1, node_2]
        self.phi = phi

        # Check that the nodes are valid
        if not isinstance(node_1, Node) or not isinstance(node_2, Node):
            raise ValueError("node_1 and node_2 must be instances of the Node class")
        
        # Make a set of the degrees of freedom
        dof = {}
        i = 0
        for degree in self.node_1.variables.keys():
            # Add the degree to the dict if not already present
            dof[degree] = i
            i+=1

        for degree in self.node_2.variables.keys():
            if degree not in dof.keys():
                dof[degree] = i
                i+=1
            
        self.dof = len(dof)
        self.variables = dof

        # Note the global indices of the degrees of freedom
        self.global_indices = [Node.var_global_idx[d] for d in dof]

# Connect node 1 to node 2
# Compute the angle of the connection
phi = np.arctan2(node2.variables["y2"] - node1.variables["y1"], node2.variables["x2"] - node1.variables["x1"])
conn1 = NodeConnection(node1, node2, phi)

# Connect node 2 to node 3
# Compute the angle of the connection
phi = np.arctan2(node3.variables["y3"] - node2.variables["y2"], node3.variables["x3"] - node2.variables["x2"])
conn2 = NodeConnection(node2, node3, phi)

print(conn2.variables)
print(conn2.global_indices)
print(conn1.variables)
print(conn1.global_indices)

{'x2': 0, 'y2': 1, 'theta2': 2, 'x3': 3, 'y3': 4, 'theta3': 5}
[3, 4, 5, 6, 7, 8]
{'x1': 0, 'y1': 1, 'theta1': 2, 'x2': 3, 'y2': 4, 'theta2': 5}
[0, 1, 2, 3, 4, 5]


# Finite Elements

In [39]:
class FiniteElement:
    def __init__(self, E, A, L,I, rho, node_1: Node, node_2: Node, node_connection: NodeConnection):
        self.E = E
        self.A = A
        self.L = L
        self.I = I
        self.rho = rho
        
        # Check that the nodes are valid
        if not isinstance(node_1, Node) or not isinstance(node_2, Node):
            raise ValueError("node_1 and node_2 must be instances of the Node class")
        
        # Check that the node connection is valid
        if not isinstance(node_connection, NodeConnection):
            raise ValueError("node_connection must be an instance of the NodeConnection class")
        
        self.node_1 = node_1
        self.node_2 = node_2
        self.node_connection = node_connection
        self.nodes = [node_1, node_2]
        
        # Register the finite element degrees of freedom
        self.dof = node_connection.dof
        self.variables = node_connection.variables

        # Register the global indices of the degrees of freedom
        self.global_indices = node_connection.global_indices

    def local_M(self,t,x):
        l = self.L
        return (
            self.rho * self.A * self.L / 420
        )* jnp.array([
            [   140,    0,              0,              70 ,            0,                  0       ],
            [   0,      156,            22*l,           0,              54,                 -13*l   ],
            [   0,      22*l,           4*l**2,         0,              13*l,               -3*l**2 ],
            [   70,     0,              0,              140,            0,                  0       ],
            [   0,      54,             13*l,           0,              156,                -22*l   ],
            [   0,      -13*l,          -3*l**2,        0,              -22*l,              4*l**2  ]
        ])
    
    def local_K(self,t,x):

        a1 = self.E*self.A/self.L
        a2 = self.E*self.I/self.L**2
        a3 = self.E*self.I/self.L**3

        return jnp.array(
            [
                [a1,    0,          0,          -a1,    0,          0       ],
                [0,     12*a3,      6*a2,       0,      -12*a3,     6*a2    ],
                [0,     6*a2,       4*a1,       0,      -6*a2,      2*a1    ],
                [-a1,   0,          0,          a1,     0,          0       ],
                [0,     -12*a3,     -6*a2,      0,      12*a3,      -6*a2   ],
                [0,     6*a2,       2*a1,       0,      -6*a2,      4*a1    ]
            ]
        )
    
    def local_C(self,t,x):
        return jnp.array([
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0]
        ])
    
    @property
    def R(self):
        phi = self.node_connection.phi
        cs = jnp.cos(phi)
        sn = jnp.sin(phi)

        return jnp.array([
            [cs, sn, 0, 0, 0, 0],
            [-sn, cs, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0],
            [0, 0, 0, cs, sn, 0],
            [0, 0, 0, -sn, cs, 0],
            [0, 0, 0, 0, 0, 1]
        ])
    
    # @partial(jit, static_argnums=(0))
    def M(self,t,x):
        return self.R.T @ self.local_M(t,x) @ self.R
    
    # @partial(jit, static_argnums=(0))
    def K(self,t,x):
        return self.R.T @ self.local_K(t,x) @ self.R

    # @partial(jit, static_argnums=(0)) 
    def C(self,t,x):
        return self.R.T @ self.local_C(t,x) @ self.R
    
    def __str__(self):
        return f"Finite Element with {self.dof} degrees of freedom.\n\tNode 1: {self.node_1}\n\tNode 2: {self.node_2}"
    
# Create two finite elements
E = 235e9
A = np.pi * (0.3**2) / 4
L = 2.0
I = 30e-2
rho = 5490

element1 = FiniteElement(E, A, L, I, rho, node1, node2, conn1)
element2 = FiniteElement(E, A, L, I, rho, node2, node3, conn2)

In [40]:
from typing import Any


class BoundaryCondition:
    """
    Define a boundary condition for a node. boundary conditions can be of the following types:
    - Joint: The node is fixed in space
    - Roller: The node is fixed in one direction
    
    """
    def __init__(self, nodes: list[Node]):
        raise NotImplementedError("BoundaryCondition is an abstract class and cannot be instantiated")

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        raise NotImplementedError("BoundaryCondition is an abstract class and cannot be instantiated")
    
    def __str__(self) -> str:
        raise NotImplementedError("BoundaryCondition is an abstract class and cannot be instantiated")
    
class JointBoundaryCondition(BoundaryCondition):
    def __init__(self, nodes: list[Node]):
        self.nodes = nodes
        self.dof = len(nodes[0].variables.keys())
        self.variables = nodes[0].variables
        self.global_indices = [Node.var_global_idx[d] for d in nodes[0].variables.keys()]

    def __call__(self, t: float, x: jnp.ndarray) -> jnp.ndarray:
        return jnp.array([0.0 for _ in range(self.dof)])
    
    def __str__(self) -> str:
        return f"Joint boundary condition for node {self.nodes[0].index}"
    
class RollerBoundaryCondition(BoundaryCondition):
    def __init__(self, nodes: list[Node], direction: str):
        self.nodes = nodes
        self.direction = direction
        self.dof = 1
        self.variables = {direction: 0.0}
        self.global_indices = [Node.var_global_idx[direction]]

    def __call__(self, t: float, x: jnp.ndarray) -> jnp.ndarray:
        return jnp.array([0.0])
    
    def __str__(self) -> str:
        return f"Roller boundary condition for node {self.nodes[0].index} in the {self.direction} direction"
    
class FixedBoundaryCondition(BoundaryCondition):
    def __init__(self, nodes: list[Node], direction: str):
        self.nodes = nodes
        self.direction = direction
        self.dof = 1
        self.variables = {direction: 0.0}
        self.global_indices = [Node.var_global_idx[direction]]

    def __call__(self, t: float, x: jnp.ndarray) -> jnp.ndarray:
        return jnp.array([0.0])
    
    def __str__(self) -> str:
        return f"Fixed boundary condition for node {self.nodes[0].index} in the {self.direction} direction"

In [66]:
class NodalSystem:
    """
    An nodal system represents a collection of finite elements that are connected to each other. 
    """
    def __init__(self, elements: list[FiniteElement], boundary_conditions: list[BoundaryCondition] = []):
        self.elements = elements
        
        # Register the degrees of freedom using a set
        dof = {}
        for element in elements:
            for variable in element.variables:
                dof[variable] = Node.get_global_idx(variable)
        self.variables = dof
        self.dof = len(dof)

    @partial(jit, static_argnums=(0))
    def M(self,t, x):
        # Construct the M matrix by adding the local M matrices. 
        # The global M matrix is a block matrix of the local M matrices
        # The matrix should be sparse
        M = jnp.zeros((self.dof, self.dof))

        # To construct the matrix loop over all the elements. 
        for element in self.elements:
            # Get each elements K matrix expressed in the global coordinate system
            sub_M =  element.M(t,x)
            for local_row, dof in enumerate(element.node_connection.variables):
                # Set the mask true for each idx in the Nodes global indices
                global_row = Node.get_global_idx(dof)
                columns = jnp.array([ i  for i in element.global_indices])          
                M = M.at[global_row, columns].add(sub_M[local_row, :])
        # print(f"M: ")
        # print(M)
        return M
    
    @partial(jit, static_argnums=(0))
    def K(self, t, x):
        # Construct the K matrix by adding the local K matrices. 
        # The global K matrix is a block matrix of the local K matrices
        # The matrix should be sparse
        K = jnp.zeros((self.dof, self.dof))

        # To construct the matrix loop over all the elements. 
        for element in self.elements:
            # Get each elements K matrix expressed in the global coordinate system
            sub_K =  element.K(t,x)
            for local_row, dof in enumerate(element.node_connection.variables):
                # Set the mask true for each idx in the Nodes global indices
                global_row = Node.get_global_idx(dof)
                columns = jnp.array([ i  for i in element.global_indices])
                K = K.at[global_row, columns].add(sub_K[local_row,:])
        # print(f"K: ")
        # print(K)
        return K
    
    @partial(jit, static_argnums=(0))
    def C(self,t,x):
        # Construct the C matrix by adding the local C matrices. 
        # The global C matrix is a block matrix of the local C matrices
        # The matrix should be sparse
        C = jnp.zeros((self.dof, self.dof))

        # To construct the matrix loop over all the elements. 
        for element in self.elements:
            # Get each elements C matrix expressed in the global coordinate system
            sub_C =  element.C(t,x)
            for node in element.nodes:
                for local_row, dof in enumerate(node.variables.keys()):
                    # Set the mask true for each idx in the Nodes global indices
                    global_row = Node.get_global_idx(dof)                    
                    columns = jnp.array([ i  for i in element.global_indices])               
                    C = C.at[global_row, columns].add(sub_C[local_row, :])
        # print(f"C:")
        # print(C)
        return C        
    
# Create the system
sys = NodalSystem([element1, element2])

def f_ext(t: float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array([
        jnp.cos(5*t) * x[0],                       # x1
        0.0,                                # y1
        0.0,                                # theta1
        -jnp.cos(5*t)/2 * x[3],                    # x2
        0.0,                                # y2
        0.0,                                # theta2
        -jnp.cos(5*t)/2 * x[6],                    # x3    
        0.0,                                # y3
        0.0                                 # theta3,
    ])

# Model

In [67]:
system = SecondOrderSystem(sys.M, sys.C, sys.K, f_ext)
x0 = jnp.zeros(sys.dof*2)
# x0 = x0.at[0].set(0.1)

A = system.linearize(0.0, x0)[0]
M = sys.M(0.0, x0)
C = sys.C(0.0, x0)
K = sys.K(0.0, x0)

# # Plot the matrices
# ax = plt.matshow(A)
# plt.title("A matrix")
# # Add text to each cell
# for i in range(sys.dof*2):
#     for j in range(sys.dof*2):
#         num = A[i,j]
#         if num == 0:
#             continue
#         # Convert num to scientific notation with 3 decimal points
#         num = f"{num:.1e}"
#         plt.text(j, i, num, ha='center', va='center', color='white', fontsize=4)

# plt.show()

# ax = plt.matshow(M)
# plt.title("M matrix")
# # Add text to each cell
# for i in range(sys.dof*2):
#     for j in range(sys.dof*2):
#         num = M[i,j]
#         if num == 0:
#             continue
#         # Convert num to scientific notation with 3 decimal points
#         num = f"{num:.1e}"
#         plt.text(j, i, num, ha='center', va='center', color='white', fontsize=4)

# plt.show()

# ax = plt.matshow(C)
# plt.title("C matrix")
# # Add text to each cell
# for i in range(sys.dof*2):
#     for j in range(sys.dof*2):
#         num = C[i,j]
#         if num == 0:
#             continue
#         # Convert num to scientific notation with 3 decimal points
#         num = f"{num:.1e}"
#         plt.text(j, i, num, ha='center', va='center', color='white', fontsize=4)

# plt.show()

# ax = plt.matshow(K)
# plt.title("K matrix")
# # Add text to each cell
# for i in range(sys.dof*2):
#     for j in range(sys.dof*2):
#         num = K[i,j]
#         if num == 0:
#             continue
#         # Convert num to scientific notation with 3 decimal points
#         num = f"{num:.1e}"
#         plt.text(j, i, num, ha='center', va='center', color='white', fontsize=4)

# plt.show()

system.eigenvalues(0.0, x0)

Array([-7.5275992e+04+0.0000000e+00j,  8.0566406e-03+8.2213742e+04j,
        8.0566406e-03-8.2213742e+04j, -5.9605230e+04+0.0000000e+00j,
       -3.5400391e-03+6.7198289e+04j, -3.5400391e-03-6.7198289e+04j,
        7.5275930e+04+0.0000000e+00j,  5.9605277e+04+0.0000000e+00j,
       -1.3427734e-03+1.1332051e+04j, -1.3427734e-03-1.1332051e+04j,
        2.6321411e-04+5.6660239e+03j,  2.6321411e-04-5.6660239e+03j,
        5.7983398e-03+1.9356770e+04j,  5.7983398e-03-1.9356770e+04j,
       -7.0289154e+00+5.3583355e+00j, -7.0289154e+00-5.3583355e+00j,
        7.0313339e+00+5.3407125e+00j,  7.0313339e+00-5.3407125e+00j],      dtype=complex64)

In [68]:
jnp.linalg.inv(sys.elements[0].M(0.0, x0))

Array([[ 0.00515377,  0.        ,  0.        , -0.00257689,  0.        ,
         0.        ],
       [ 0.        ,  0.02061507, -0.07730651,  0.        , -0.00515375,
        -0.03865322],
       [ 0.        , -0.07730649,  0.3865325 ,  0.        ,  0.03865318,
         0.2705726 ],
       [-0.00257689,  0.        ,  0.        ,  0.00515377,  0.        ,
         0.        ],
       [-0.        , -0.00515375,  0.03865319, -0.        ,  0.02061507,
         0.07730648],
       [ 0.        , -0.03865319,  0.2705725 ,  0.        ,  0.07730648,
         0.3865324 ]], dtype=float32)

In [69]:
dt0 = 1e-5
t0 = 0.0
t_end = 60

# test_all_integrators(system, x0, t0, t_end, dt0, compare_with_scipy = False)

# # Newmark integrator
newmark = NewmarkIntegrator(dt0, system, gamma=0.5, beta=0.25)

t_data, x_data = newmark.simulate(x0, t0, t_end)
# t_data, x_data = gauss_legendre.simulate(x0, t0, t_end)

In [70]:
import matplotlib.pyplot as plt

clip = 1000


fig, ax = plt.subplots(3, 1, figsize=(10, 10))

# Clip the data to only sample 1000 points
clip = jnp.maximum(1, int(len(t_data) / 1000))

ax[0].set_title("Displacement of node 1")
ax[0].plot(t_data[::clip], x_data[:,0][::clip], label="x1")
ax[0].plot(t_data[::clip], x_data[:,1][::clip], label="y1")
ax[0].plot(t_data[::clip], x_data[:,2][::clip], label="theta1")
ax[0].legend()

ax[1].set_title("Displacement of node 2")
ax[1].plot(t_data[::clip], x_data[:,3][::clip], label="x2")
ax[1].plot(t_data[::clip], x_data[:,4][::clip], label="y2")
ax[1].plot(t_data[::clip], x_data[:,5][::clip], label="theta2")
ax[1].legend()

ax[2].set_title("Displacement of node 3")
ax[2].plot(t_data[::clip], x_data[:,6][::clip], label="x3")
ax[2].plot(t_data[::clip], x_data[:,7][::clip], label="y3")
ax[2].plot(t_data[::clip], x_data[:,8][::clip], label="theta3")
ax[2].legend()